# 神经网络的学习

In [1]:
import os, sys
import numpy as np
sys.path.append(os.pardir)

import common.functions as functions
import common.gradient as gradient

In [2]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
functions.mean_squared_error(np.array(y), np.array(t))

0.09750000000000003

In [3]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
r1 = functions.cross_entropy_error(np.array(y), np.array(t))
r2 = functions.cross_entropy_error_v2(np.array(y), np.array(t))
r3 = functions.cross_entropy_error_v3(np.array(y), np.array(t))

r1, r2, r3

(0.510825457099338, 0.510825457099338, 23.718987110506905)

In [4]:
np_y = np.array(y)
np_y.ndim
np_y.shape
new_y = np_y.reshape(1, np_y.size)
np_y.shape, new_y.shape

((10,), (1, 10))

In [5]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

# 训练集，训练标签，测试集，测试标签
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
x_train.shape, t_train.shape

((60000, 784), (60000,))

In [6]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [7]:
func = functions.squard

# numerical_diff_pure 中误差项太小，引起了计算机计算精度的丢失，出来的结果是0，导致计算出错
functions.numerical_diff(func, 2), functions.numerical_diff_pure(func, 2)

(4.000000000004, 0.0)

In [8]:
gradient.numerical_gradient(functions.function_2, np.array([0.0, 2.0]))

array([0., 4.])

In [39]:
# 两层神经网络
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params={}
        self.grad={}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)

        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)


    def predict(self, x: "numpy.ndarray")->"numpy.ndarray":
        params = self.params
        W1, W2 = params['W1'], params['W2']
        b1, b2 = params['b1'], params['b2']


        r1 = functions.sigmoid(np.dot(x, W1) + b1)
        r2 = functions.softmax(np.dot(r1, W2) + b2)

        result = r2
        return result


    def loss(self,x: "numpy.ndarray", t: "numpy.ndarray"):
        y_h = self.predict(x) # 预测值
        loss = functions.cross_entropy_error(y_h, t)
        return loss


    def accuracy(self, x, t):
        y = self.predict(x)
        # axis = 1, 求的是每一行的最大值
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy


    def numerical_gradient(self,x, t):
        loss_func = lambda W: self.loss(x, t)

        W1, W2, b1, b2 = self.params['W1'], self.params['W2'], self.params['b1'], self.params['b2']
        self.grad['W1'] = gradient.numerical_gradient(loss_func, W1)
        self.grad['W2'] = gradient.numerical_gradient(loss_func, W2)
        self.grad['b1'] = gradient.numerical_gradient(loss_func, b1)
        self.grad['b2'] = gradient.numerical_gradient(loss_func, b2)


        return self.grad
        

In [46]:
# 使用mini-batch训练神经网络

import numpy as np
from dataset.mnist import load_mnist
# from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label = True)
train_loss_list = []
# mnist 是 28 * 28的图片，flatten压平后就是 784维度
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

print(x_train.shape, t_train.shape, x_test.shape, t_test.shape)

iters_num = 100
data_count = x_train.shape[0]
select_num = data_count / iters_num



learn_rate = 0.05

for i in range(iters_num):
    print(f"====iter {i} start=====")
    
    params = network.params
    batch_mask = np.random.choice(int(select_num), data_count)

    batch_x_train = x_train[batch_mask]
    batch_t_train = t_train[batch_mask]

    loss = network.loss(batch_x_train, batch_t_train)

    train_loss_list.append(loss)

    grad = network.numerical_gradient(batch_x_train, batch_t_train)

    # 梯度
    W1_g, W2_g, b1_g, b2_g = grad['W1'], grad['W2'], grad['b1'], grad['b2']
    # 老的参数
    W1, W2, b1, b2 = params['W1'], params['W2'], params['b1'], params['b2']
    # 以梯度为指南针，更新参数。
    # 目的是让loss减小，参数只能沿着梯度的反方向变化
    W1_n = W1 - learn_rate * W1_g
    W2_n = W2 - learn_rate * W2_g
    b1_n = b1 - learn_rate * b1_g
    b2_n = b2 - learn_rate * b2_g
    params['W1'] = W1_n
    params['W2'] = W2_n
    params['b1'] = b1_n
    params['b2'] = b2_n
    network.params = params
    print(f"iter count: {i}, loss: {loss}")

(60000, 784) (60000, 10) (10000, 784) (10000, 10)
====iter 0 start=====


KeyboardInterrupt: 

In [ ]:
# 评价模型
trained_network = network

test_acc = trained_network.accuracy(x_train, t_train)
test_loss = trained_network.loss(x_train, t_train)

test_acc, test_loss

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
# from two_layer_net import TwoLayerNet

# 读入数据
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000  # 适当设定循环的次数
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 计算梯度
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch)
    
    # 更新参数
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

# 绘制图形
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

AttributeError: 'TwoLayerNet' object has no attribute 'gradient'